In [25]:
import numpy as np
import types
import cv2 as cv
# cv.imshow() -> to display image

In [26]:
#function to convert any type of data into binary
def messageToBinary(message):
    if(type(message) == str):
        temp = ''
        for i in message:
            temp = temp + format(ord(i), "08b")
        return temp
    elif(type(message) == bytes or type(message) == np.ndarray):
        print(message)
        temp = []
        for i in message:
            temp.append(format(i, "08b"))
        return temp
    elif(type(message) == int or type(message) == np.uint8):
        return format(message, "08b")
    else:
        raise TypeError('Input Type not supported')


In [27]:
a = np.array([13, 2, 5])
print(messageToBinary(a))

[13  2  5]
['1101', '0010', '0101']
